In [1]:
import pickle as pic
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import cPickle
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV as gsearch
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier as forest
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.cross_validation import KFold
import nltk.data
%matplotlib inline
nltk.download('stopwords')

C:\Users\Keenan\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Keenan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def save_obj(obj, name):
    with open(name, 'wb') as f:
        pic.dump(obj, f, pic.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open(name, 'rb') as f:
        return pic.load(f)

In [11]:
five_k = load_obj('data/tmdb_df_5k.pickle')

In [12]:
def df_overview_clean(df):
    ret_df = df.copy()
    return ret_df[['title', 'genres', 'overview']]

In [13]:
five_k_over = df_overview_clean(five_k)

In [14]:
classes = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
           'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
            'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

In [15]:
def genre_extract(genre_dic):
    ret_genres = []
    for dic in genre_dic:
        ret_genres.append(dic['name'])
    return ret_genres

def assign_genre_vector(row_genres, all_classes):
    ret_list = []
    genre_list = genre_extract(row_genres)
    for genre in all_classes:
        if genre in genre_list:
            ret_list.append(1)
        else:
            ret_list.append(0)
    return ret_list           

In [16]:
five_k_vec = pd.DataFrame(columns=['title', 'genres', 'overview', 'genre_vec', 'all_genres'])
for row in zip(five_k_over['title'], five_k_over['genres'], five_k_over['overview']):
    genre_vec = assign_genre_vector(row[1], classes)
    genres = genre_extract(row[1])
    
    #had to throw this in for the vectorization process-later
    overview = row[2]
    if row[2] == None or not row[2]:
        overview = 'a'
    five_k_vec = five_k_vec.append({'title':row[0], 'genres':row[1], 'overview':overview, 'genre_vec':genre_vec, 'all_genres':genres}, ignore_index=True)

In [17]:
#set our vectorization
vectorizer = CountVectorizer(stop_words=stopwords.words("english"),
    max_df = 0.9, 
    min_df = 5, 
    dtype=np.float32 )

corpus = five_k_vec['overview'].values
X = vectorizer.fit_transform(corpus).toarray()
ys = MultiLabelBinarizer().fit_transform(five_k_vec['all_genres'])

In [18]:
#http://stackoverflow.com/questions/36523558/multi-class-logistic-regression-in-scikit-learn
#And you seem to be looking for multilabel (as for multiclass labels should be 1-dim). 
#Currently, in sklearn, the only methods supporting multilabel are:
#Decision Trees, Random Forests, Nearest Neighbors, Ridge Regression.
#have to use onevsrest classification

In [19]:
def cross_val_mod(X_train, y_train, mod, parameters, folds = 5):
    grid_clf = gsearch(mod, parameters, cv=folds)
    grid_clf.fit(X_train, y_train)
    return (grid_clf.best_estimator_, grid_clf.best_params_)

In [22]:
#Log Reg without CV
X_train,X_test,y_train,y_test = train_test_split(X,ys,test_size=0.3)

oneVsResClassifier = OneVsRestClassifier(LogReg()).fit(X_train, y_train)

oneVsResClassifier.score(X_test, y_test)

0.15533333333333332

In [8]:
#forest without CV
X_train,X_test,y_train,y_test = train_test_split(X,ys,test_size=0.3)

forest_mod = forest()
forest_mod.fit(X_train, y_train)

forest_mod.score(X_test, y_test)

NameError: name 'X' is not defined

These only perform as well as the baseline, probably because they are over classifying the most popular classes

In [30]:
#Log Reg with CV
X_train,X_test,y_train,y_test = train_test_split(X,ys,test_size=0.3)

all_c = np.power(10.0, range(-3, 3))

params = {'estimator__C':all_c}

cv_mod = cross_val_mod(X_train, y_train, OneVsRestClassifier(LogReg(class_weight='balanced')), params, 5)

print cv_mod[0].score(X_test, y_test)
print cv_mod[1]

0.146
{'estimator__C': 0.10000000000000001}


In [20]:
#Log Reg with optimized parameters
X_train,X_test,y_train,y_test = train_test_split(X,ys,test_size=0.3)

oneVsResClassifier = OneVsRestClassifier(LogReg(C=.1)).fit(X_train, y_train)

oneVsResClassifier.score(X_test, y_test)

0.13600000000000001

In [25]:
model_dict = {}
model_dict['model'] = oneVsResClassifier
model_dict['X_train'] = X_train
model_dict['y_train'] = y_train
model_dict['X_test'] = X_test
model_dict['y_test'] = y_test
model_dict['labels_names'] = classes

with open('models/logreg_text_model.pickle', 'wb') as handle:
    cPickle.dump(model_dict, handle, protocol = cPickle.HIGHEST_PROTOCOL)

In [20]:
#forest with CV
X_train,X_test,y_train,y_test = train_test_split(X,ys,test_size=0.3)

params = {'n_estimators': [5, 10, 15, 20],
          'max_depth': [2, 5, 7, 9]}
cv_mod = cross_val_mod(X_train, y_train, forest(), params, 10)

print cv_mod[0].score(X_test, y_test)
print cv_mod[1]

KeyboardInterrupt: 

In [28]:
#forest with optimized parameters
X_train,X_test,y_train,y_test = train_test_split(X,ys,test_size=0.3)

#forest_mod = forest(n_estimators=5, max_depth=9)
forest_mod = forest()
forest_mod.fit(X_train, y_train)

forest_mod.score(X_test, y_test)

0.128

In [29]:
model_dict = {}
model_dict['model'] = forest_mod
model_dict['X_train'] = X_train
model_dict['y_train'] = y_train
model_dict['X_test'] = X_test
model_dict['y_test'] = y_test
model_dict['labels_names'] = classes

with open('models/forest_text_model.pickle', 'wb') as handle:
    cPickle.dump(model_dict, handle, protocol = cPickle.HIGHEST_PROTOCOL)

In [7]:
d = load_obj('models/sgd_text_model.pickle')
print d
print classification_report(d['y_test'], d['model'].predict(d['X_test']), target_names = d['labels_names'])

{'X_test': <9417x31396 sparse matrix of type '<type 'numpy.float64'>'
	with 504926 stored elements in Compressed Sparse Row format>, 'X_train': <28251x31396 sparse matrix of type '<type 'numpy.float64'>'
	with 1514927 stored elements in Compressed Sparse Row format>, 'labels_names': array([u'Action', u'Adventure', u'Animation', u'Comedy', u'Crime',
       u'Documentary', u'Drama', u'Family', u'Fantasy', u'Horror',
       u'Mystery', u'Romance', u'Sci-Fi', u'Short', u'Thriller'], dtype=object), 'y_train': array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'model': GridSearchCV(cv=None, error_score='raise',
       estimator=OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n

In [9]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(d['y_test'], d['model'].predict(d['X_test']))

ValueError: multilabel-indicator is not supported

In [6]:
d2 = load_obj('models/forest_text_model.pickle')
print d2
print classification_report(d2['y_test'], d2['model'].predict(d2['X_test']), target_names = d2['labels_names'])

{'X_test': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), 'X_train': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), 'labels_names': [array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1])], 'y_train': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0